<a href="https://colab.research.google.com/github/Angelawork/AI4Good_LiteraLingo_M3Project/blob/main/Figurative-detection%20roberta%20by%20Angela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should pr

In [ ]:
import pandas as pd
df=pd.read_csv('/content/classification.csv')[:1000].sample(frac=1).reset_index(drop=True)

texts = df['Figurative sentence'].tolist()
labels = df['Type'].tolist()

inputs = tokenizer.batch_encode_plus(
    df['Figurative sentence'].tolist(),
    padding="longest",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [ ]:
df

,Type,Unnamed: 1,Figurative sentence
0,Literal meaning,NaN,"You make it sound like the bad old days, mumbl..."
1,Literal meaning,NaN,"Thin as smoke now, the spiral drifted out into..."
2,Literal meaning,NaN,I guess I always found meaning in what I learn...
3,Literal meaning,NaN,"It's my job now to go outside, have fun and ma..."
4,Idiom,NaN,Going to the chief of the Makaho in mourning g...
...,...,...,...
995,Idiom,NaN,I've probably always been able to make sense o...
996,Literal meaning,NaN,"Then why not let me try to contact them, meet ..."
997,Idiom,NaN,"Your mother's not in the best of tempers, is s..."
998,Idiom,NaN,This is all a pack of lies.


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Type'])

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should pr

In [ ]:
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted_labels = torch.max(logits, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)

    # Print training statistics
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct_predictions / total_predictions
    print(f"Epoch {epoch+1} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy*100:.2f}%")

Epoch 1 - Loss: 1.1876 - Accuracy: 41.80%
Epoch 2 - Loss: 0.7626 - Accuracy: 64.50%
Epoch 3 - Loss: 0.4160 - Accuracy: 83.00%
Epoch 4 - Loss: 0.1306 - Accuracy: 97.00%
Epoch 5 - Loss: 0.0951 - Accuracy: 97.20%


In [ ]:
df2=pd.read_csv('/content/classification.csv')[1010:].sample(frac=1).reset_index(drop=True)

texts = df2['Figurative sentence'].tolist()
labels = df2['Type'].tolist()

inputs = tokenizer.batch_encode_plus(
    df2['Figurative sentence'].tolist(),
    padding="longest",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
labels = label_encoder.fit_transform(df2['Type'])
eval_dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], torch.tensor(labels))

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, 1)
            print(labels)
            print(predicted_labels)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy

eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)
accuracy = evaluate_model(model, eval_dataloader)
print("Accuracy:", accuracy)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 1, 0, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 1, 1, 1, 1, 3, 1, 1, 0, 1, 1, 3, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 0, 3, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([0, 0, 0])
tensor([3, 1, 1])
Accuracy: 0.1927710843373494


In [ ]:
model.save_pretrained('RobertaModel_figurative detection_Angela')
tokenizer.save_pretrained('RobertaTokenizer_figurative detection_Angela')
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil

source_path = "/content/RobertaModel_figurative detection_Angela/pytorch_model.bin"  # Replace with the actual path to your file
target_path = "/content/drive/MyDrive/pytorch_model.bin"  # Replace with the desired path in your Google Drive

shutil.copyfile(source_path, target_path)

'/content/drive/MyDrive/pytorch_model.bin'